Manipulate the project in VS Code
Open the project and print some information (10 pnts)

In [4]:
import arcpy
import os
pathname = r"C:\Users\tamai\OneDrive - Louisiana State University\Fall 2024\GEOG 4057\Assignments\Eighth Assignment\Austin.aprx"
aprx = arcpy.mp.ArcGISProject(os.path.join(pathname))
print(aprx.defaultGeodatabase)

C:\Users\tamai\OneDrive - Louisiana State University\Fall 2024\GEOG 4057\Assignments\Eighth Assignment\Austin_Data.gdb


Save a copy of the project (10 pnts)

In [6]:
import arcpy
import os

pathname = r"C:\Users\tamai\OneDrive - Louisiana State University\Fall 2024\GEOG 4057\Assignments\Eighth Assignment"
aprx = arcpy.mp.ArcGISProject(os.path.join(pathname, 'Austin.aprx'))
aprx.saveACopy(os.path.join(pathname, 'Austin_Copy.aprx'))


Examine the new project file (10 pnts)
Open the new project file you created in ArcGIS Pro to check the items in the project
Is everything the same as the original project? (Compare the difference in the file sizes) and why?
Answer: 

The copied project file is typically slightly smaller than the original. This size difference occurs because saveACopy omits certain temporary items, such as undo/redo histories and certain cached data, which can accumulate during regular use and inflate the original file's size.

Use the following code to print a list of maps in the project (10 pnts)

In [1]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print(m.name)
    print(m.mapUnits)
del aprx

Downtown
Foot_US
Region
Foot_US


What does del do in the last line? Was the project file deleted?
Answer: The del aprx line removes the aprx variable from memory, freeing up resources by effectively "deleting" the variable reference to the project in the script. It does not delete the actual project file (Austin.aprx) on disk.

Use the following code to change the name of a map (10 pnts)

In [2]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Region")[0]
m.name = "County"
del aprx

Now, the map name is changed into county

Use the following code to list the layers in a map (10 pnts)

In [3]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
maps = aprx.listMaps()
for m in maps:
    print("Map: " + m.name)
    lyrs = m.listLayers()
    for lyr in lyrs:
        print(lyr.name)
del aprx

Map: Downtown
trees
parks
base
Topographic
Map: County
facilities
hospitals
parks
Topographic


Additionally, use the following code to print if a layer is a basemap or a feature layer

In [4]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
lyrs = m.listLayers()
for lyr in lyrs:
    if lyr.isBasemapLayer:
        print(lyr.name + " is a basemap layer")
    if lyr.isFeatureLayer:
        print(lyr.name + " is a feature layer")
del aprx


trees is a feature layer
parks is a feature layer
base is a feature layer
Topographic is a basemap layer


Change the basemap (5 pnts)
Use the following code to change the basemap of Downtown

In [5]:
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.listMaps("Downtown")[0]
m.addBasemap("Light Gray Canvas")

Answer: Now the base map for "Downtown" is changed into "Light Gray Canvas"

Work with layers
Modify layer symbology (5 pnts)
Run the following code in a block to change the symbology of layers

In [6]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
green = {"RGB": [100, 175, 0, 100]}
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = green
    lyr.symbology = sym

In [ ]:
What type is the 'green' variable? What do the numbers in the list mean? If you want to show a brown color, what numbers you would fill it with? Change it in the block and confirm the color is changed to brown.

The green variable is a dictionary where the key "RGB" specifies a color in terms of RGBA (Red, Green, Blue, and Alpha/opacity) values. In this list:

First number: Red component (0–255).
Second number: Green component (0–255).
Third number: Blue component (0–255).
Fourth number: Alpha (opacity) value (0 for fully transparent, 255 for fully opaque).
In this case, green = {"RGB": [100, 175, 0, 100]} gives a green color with 39% opacity (100/255).

In [7]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps("Downtown")[0]
lyr = m.listLayers("parks")[0]
sym = lyr.symbology
brown = {"RGB": [139, 69, 19, 100]}  # Brown color with partial opacity
if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    sym.renderer.symbol.color = brown
    lyr.symbology = sym


The above code block was used to turn thr layer into brown

Work with a layout
Add a layout to the project ((5 pnts))
Run the following code to create a layout and insert the map "Downtown" in the layout

In [8]:
m = aprx.listMaps("Downtown")[0]

lyt = aprx.createLayout( 11,8.5, 'INCH', 'New Layout with Rectangles')

def MakeRec_LL(llx, lly, w, h):
    xyRecList = [[llx, lly], [llx, lly+h], [llx+w,lly+h], [llx+w,lly], [llx,lly]]
    array = arcpy.Array([arcpy.Point(*coords) for coords in xyRecList])
    rec = arcpy.Polygon(array)
    return rec

mf = lyt.createMapFrame(MakeRec_LL(0.5,0.5,10,7.5), m, "New Map Frame")

What is the size of the layout in inches?
What is the size of the map frame?
Comparing the layout and map frames you created manually, what are the advantages of creating it in Python?


Answer: In the above code, the layout is set to 11 inches wide by 8.5 inches high, as specified in aprx.createLayout(11, 8.5, 'INCH', 'New Layout with Rectangles').

The map frame is defined by calling MakeRec_LL(0.5, 0.5, 10, 7.5), which creates a 10-inch wide by 7.5-inch high frame, positioned 0.5 inches from the bottom-left corner of the layout.

Add layout elements (5 pnts)
Run the following code to create a north arrow and a scale bar

In [10]:
#Create a north arrow
naStyle = aprx.listStyleItems('ArcGIS 2D', 'North_Arrow', 'Compass North 1')[0]
na = lyt.createMapSurroundElement(arcpy.Point(9.5,7.5), 'North_Arrow', mf,
                                      naStyle, "Compass North Arrow")
na.elementWidth = 0.5  

#Create a scale bar
sbName = 'Double Alternating Scale Bar 1 Metric'
sbStyle = aprx.listStyleItems('ArcGIS 2D', 'Scale_bar', sbName)[0]
sbEnv = MakeRec_LL(5.5, 0.1, 4, 0.5)
sb = lyt.createMapSurroundElement(sbEnv, 'Scale_bar', mf, sbStyle, 'New Scale Bar')

Export the layout to a pdf (10 pnts)
Run the following code to export the layout to a pdf file

In [13]:
import os
import os
pathname = r"C:\Users\tamai\OneDrive - Louisiana State University\Fall 2024\GEOG 4057\Assignments\Eighth Assignment"


lyt.exportToPDF(os.path.join(pathname, 'downtown.pdf'))

'C:\\Users\\tamai\\OneDrive - Louisiana State University\\Fall 2024\\GEOG 4057\\Assignments\\Eighth Assignment\\downtown.pdf'